In [1]:
# Chase Abram
using Pkg
# Pkg.add("DataFrames")
using CSV
using DataFrames
using ForwardDiff
using LinearAlgebra
using Optim

using Random

# Load data
df = DataFrame()
df = CSV.read("psetOne.csv", DataFrame)

In [8]:
# names(df)

# Get market t = 17
df17 = df[df."Market" .== 17, :]

# Store variables for market 17
X = convert(Matrix, df17[:, filter(x -> x in ["Constant", "EngineSize", "SportsBike", "Brand2", "Brand3"], names(df17))])
Br = convert(Matrix, df17[:, filter(x -> x in ["Brand2", "Brand3"], names(df17))])
Br = Bool.(Br)
p_given = df17."Price"

7×2 BitArray{2}:
 0  0
 0  0
 1  0
 1  0
 0  1
 0  1
 0  1

In [59]:
# function u_bar_jt(delta, sigma, X, zeta)
#     return delta .+ X*sigma*zeta
# end

# function Prij(u_bar)
#     return exp.(u_bar)./sum(exp.(u_bar))
# end

# Why is this jumping to 0? (prices are going negative and massive)

# Shares
function s_logit(p, alpha, beta, X, xi)
    
    # Compute numerators for each s_j
    nums = exp.(X*beta - alpha.*p + xi)
    
    return nums./(1 + sum(nums))
end

# Jacobian of shares
function s_logit_jac(p, alpha, beta, X, xi)
    # Get shares
    s = s_logit(p, alpha, beta, X, xi)
    
    # Jacobian is the outer product times alpha in this case...
    jac = alpha .* s * s'
    
    # ... except for an extra term on the diag
    for j in 1: length(p)
        jac[j,j] -= alpha*s[j]
    end
    
    return jac
end

# Hessian of shares (n by n by n tensor)
function s_logit_hess(p, alpha, beta, X, xi)
    
    # Initialize
    hess = zeros(length(p), length(p), length(p))
    
    # Shares
    s = s_logit(p, alpha, beta, X, xi)

    # Jacobian of shares
    sj = s_logit_jac(p, alpha, beta, X, xi)
    
    # Fill in
    for j in 1:length(p)
        for k in 1:length(p)
            for l in 1:length(p)
                
                if j == k
                    # on diag of jacobian
                    hess[j,k,l] = alpha*sj[l,j]*(2*s[j] - 1)
                else
                    # off diag of jacobian
                    hess[j,k,l] = alpha*(sj[l,j]*s[k] + s[j]*sj[l,k])
                end
            end
        end
    end
    
    return hess
end

# Ownership structure
function Omega_star(br)
    
    # Initialize
    Om = zeros(size(br,1),size(br,1))
    
    for i in 1:size(Om,1)
        for j in 1:size(Om,2)
            if br[i,1] && br[j,1]
                # Brand2
                Om[i,j] = 1
            elseif br[i,2] && br[j,2]
                # Brand3
                Om[i,j] = 1
            elseif (!br[i,1]&& !br[j,1]) && (!br[i,2]&& !br[j,2])
                # Brand1
                Om[i,j] = 1
            end
        end
    end
    
    return Om
end

# Omega in Nevo (2001)
function Omega(p, alpha, beta, X, xi, br)
    return Omega_star(br) .* -s_logit_jac(p, alpha, beta, X, xi)
end

# Fixed point expression (want zero at FP)
function fp_logit(p, alpha, beta, X, xi, br)
    return s_logit(p, alpha, beta, X, xi) - Omega(p, alpha, beta, X, xi, br)*p
end

# (Jacobian of Omega)*p (this is used in the gradient of the FP expression)
function jac_om_p(p, alpha, beta, X, xi, br)
    # Initialize
    jacomp = zeros(length(p), length(p))
    
    # Jacobian of shares
    sj = s_logit_jac(p, alpha, beta, X, xi)
    
    # Hessian of shares
    sh = s_logit_hess(p, alpha, beta, X, xi)
    
    # Ownership
    oms = Omega_star(br)
    
    # Fill in
    for i in 1:length(p)
        for k in 1:length(p)
#             jacomp[i,k] = sum([sh[i,j,k]*p[j]*oms[i,j] for j in 1:length(p)]) + sj[i,k]*oms[i,k]
            jacomp[i,k] = -(sh[i,:,k]' * (oms[i,:] .* p))[1] #+ oms[i,k] * sj[i,k]
        end
    end
    return jacomp
end

# Jacobian of fixed point expression
function J_fp_logit(p, alpha, beta, X, xi, br)
    
    # Jacobian of shares
    sj = s_logit_jac(p, alpha, beta, X, xi)
    
    # (Jacobian of Omega)*p
    jacomp = jac_om_p(p, alpha, beta, X, xi, br)
    
    # Omega
    om = Omega(p, alpha, beta, X, xi, br)
#     println("sj: ", sj)
#     println("jacomp: ", jacomp)
#     println("om: ", om)
    return sj - jacomp - om
end

# Solve for fixed point
function solve_fp_logit(p0, alpha, beta, X, xi, br, tol = 1e-14, maxiter = 1000)
    # Initialize price and update storage
    p = p0
    pnew = zeros(length(p0))
    
    # Initialize Jacobian
    J = zeros(length(p0), length(p0))
    
    # Iterator
    it = 0
    
    # Difference in updating
    diff = Inf
    println("p0: ", p0)
    
    # Continue until converged or maximum iterations
    while diff > tol && it < maxiter
        println("p: ", p)
        
        # Jacobian of FP expression
        J .= J_fp_logit(p, alpha, beta, X, xi, br)
        println("J: ", J)
#         println("J inv: ", inv(J))
#         println("J under fp: ", J \ fp_logit(p, alpha, beta, X, xi, br))
#         println("size(p): ", size(p))
#         println("size(J under fp): ", size(J \ fp_logit(x, alpha, beta, X, xi, br)))
        
        # Update
        pnew = (p - (J \ fp_logit(p, alpha, beta, X, xi, br)))[:,1]
        it += 1

        println("pnew: ", pnew)
        
        # Get difference
        diff = maximum(abs.(pnew - p))
        println("diff: ", diff)
        
        p = pnew
    end
    
    println("Took ", it, " iterations")
    return p
end



# Squared distance of FP expression from origin
# function fp_logit_2(p, alpha, beta, X, xi, br)
#     return (fp_logit(p, alpha, beta, X, xi, br)' * fp_logit(p, alpha, beta, X, xi, br))[1]
# end


# function fp_logit_2_jac(storage, p, alpha, beta, X, xi, br)
#     sp = s_logit(p, alpha, beta, X, xi)
#     Om = Omega_star(br).* s_logit_jac(p, alpha, beta, X, xi)
#     s_jac = s_logit_jac(p, alpha, beta, X, xi)
#     jacomp = jac_om_p(p, alpha, beta, X, xi, br)
    
#     jac = 2*(sp + Om*p)' * (s_jac + jacomp + Om)
#     for i in 1:length(p)
#         storage[i] = jac[i]
#     end
# #     storage = (sp + Om*p)' * (s_jac + jacomp + Om)
# #     storage = 0
# #     return storage
    
# end

# function fp_logit_2_jac(storage, p, alpha, beta, X, xi, br)
#     s = s_logit(p, alpha, beta, X, xi)
#     oms = Omega_star(br)
#     sj = s_logit_jac(p, alpha, beta, X, xi)
#     sh = s_logit_hess(p, alpha, beta, X, xi)
    
#     for k in 1:length(p)
#         storage[k] = (2 .* (s + (oms .* sj)*p)' * (sj[k,:] + (oms .* sh[:,:,k])*p + sj[:,k]))[1]
#     end
#     println("stor: ", storage)
# #     return storage
# end

# function fp_logit_2_jac(p, alpha, beta, X, xi, br)
#     s = s_logit(p, alpha, beta, X, xi)
#     oms = Omega_star(br)
#     sj = s_logit_jac(p, alpha, beta, X, xi)
#     sh = s_logit_hess(p, alpha, beta, X, xi)
    
#     out = zeros(length(p))
#     for k in 1:length(p)
#         out[k] = (2 .* (s + (oms .* sj)*p)' * (sj[k,:] + (oms .* sh[:,:,k])*p + sj[:,k]))[1]
#     end
# #     println("stor: ", storage)
#     return out
# end

# function basic_newton_zero(f, g, init, tol = 1e-14, maxiter = 10000)
#     x = init
#     x_new = init
#     diff = Inf
#     it = 1
# #     println("x init: ", init)
#     while diff > tol && it < maxiter
# #         println("it: ", it)
# #         println("g(x): ", g(x))
#         x_new -= 0.5.*g(x)
# #         println("x_new: ", x_new)
#         diff = maximum(abs.(x_new - x))
# #         println("diff: ", diff)
#         it += 1
#         x = x_new
#     end
#     return x
# end


# function solve_fp_logit(alpha, beta, X, xi, br, tol = 1e-14, maxit = 100)
    
#     f(x) = fp_logit_2(x, alpha, beta, X, xi, br)[1,1]
    
# #     g!(storage, x) = fp_logit_2_jac(storage, x, alpha, beta, X, xi, br)
#     g!(x) = fp_logit_2_jac(x, alpha, beta, X, xi, br)
    
#     out = basic_newton_zero(f, g!, p)
# #     Optim.Options(f_tol = tol)
# #     Optim.Options(time_limit = 10.0)
# #     opt = optimize(f, g!, p, LBFGS(), Optim.Options(time_limit = 5.0))
# #     opt = optimize(f, x -> ForwardDiff.gradient(f, p), p, BFGS(), Optim.Options(time_limit = 10.0))
# #     return opt, opt.minimizer, opt.minimum
#     return out
# end



solve_fp_logit (generic function with 3 methods)

In [72]:
p = ones(size(X,1),1) .* 1.0
# p .= p_given
# p[7] = 1
alpha = 3
beta = [1 1 2 -1 1]'


# seed maintains results across runs
rng = MersenneTwister(1234)
xi = randn(rng, size(df17,1)) .* 0.0
# println("xi: ", xi)

# println("p: ", p)
# println("delta: ", X*beta - alpha .* p + xi)


println("shares: ", s_logit(p, alpha, beta, X, xi))
# println("sum shares: ", sum(s_logit(p, alpha, beta, X, xi)))

s_logit_jac(p, alpha, beta, X, xi)
# println("shares_jac: ", s_logit_jac(p, alpha, beta, X, xi))

s_logit_hess(p, alpha, beta, X, xi)

Omega_star(Br)

# Omega(p, alpha, beta, X, xi, Br)

# fp_logit(p, alpha, beta, X, xi, Br)' * fp_logit(p, alpha, beta, X, xi, Br)
# println("fp_logit: ", fp_logit(p, alpha, beta, X, xi, Br))
fp_logit(p, alpha, beta, X, xi, Br)

# J_fp_logit(p, alpha, beta, X, xi, Br)

# out = solve_fp_logit(p, alpha, beta, X, xi, Br)
# println("out: ", out)
# println("fp_logit: ", fp_logit(out, alpha, beta, X, xi, Br))



# fp_logit_2(p, alpha, beta, X, xi, Br)
# st = zeros(length(p))
# fp_logit_2_jac(st, p, alpha, beta, X, xi, Br)
# println(st)

# jac_om_p(p, alpha, beta, X, xi, Br)

# fp_logit_jac(p, alpha, beta, X, xi, Br)


# op = solve_fp_logit(alpha, beta, X, xi, Br)
# println("op: ", op)
# println("opam: ", opam)
# println("test on fp: ", fp_logit(opam, alpha, beta, X, xi, Br))

# op = optimize(x -> (fp_logit(x, alpha, beta, X, xi, Br)' * fp_logit(x, alpha, beta, X, xi, Br))[1,1], p, LBFGS())

# pmin = op.minimizer
# println("p min: ", pmin)
# println("min: ", op.minimum)
# println("fp_logit: ", fp_logit(pmin, alpha, beta, X, xi, Br))
# df17[3, "Brand2"]

# Gradient must be wrong.
# Check whole construction and parameter choice.


shares: [0.00029341911289887894; 0.0021680902857082822; 0.00010794285928225993; 0.0007975958427155957; 0.0035745765708455634; 0.8746700481466118; 0.11837371870450338]


7×1 Array{Float64,2}:
 -0.0005846714640853634
 -0.0043201702475706635
 -0.00021559247925446672
 -0.001593024923718797
  0.0035383125997220477
  0.8657965469808248
  0.11717282091093119

In [41]:
# NM: opam: [0.3720742469890362; 0.3724406061806239; 0.3447723065789802; 
# 0.3424326539927414; 2.4893064284464703; 2.5105275013368717; 4.366510826679992]

# BFGS: opam: [0.37346117252339095; 0.37346116923202516; 5.8690207005673365; 
# 0.34160603728869476; 2.5287477985474984; 2.5287470025674392; 2.528747016448347]







In [42]:
function f1(x, y, z) 
    return (x[1] - 5 - y)^2 + (x[2] - 3 - z)^2
end

function g1(storage, x, y, z)
    storage[1] = 2*(x[1] - 5 - y)
    storage[2] = 2*(x[2] - 3 - z)
end

function g2(x,y,z)
    out = zeros(2)
    out[1] = 2*(x[1] - 5 - y)
    out[2] = 2*(x[2] - 3 - z)
    
    return out
end

function tester()
    a = 1000
    b = -1000
    f(x) = f1(x,a,b)
#     g!(storage, x) = g1(storage, x, a, b)
    g!(x) = g2(x,a,b)
    
    out = basic_newton_zero(f, g!, [-200.0, 1000.0])
    
#     opt = optimize(f, g!, [100.0, -20.0], LBFGS())
#     return opt, opt.minimizer, opt.minimum
    return out
end

tester()



2-element Array{Float64,1}:
 1005.0
 -997.0

In [70]:
p = ones(size(X,1),1) .* 1.0
# p[1] = 50
# p[2] = 30
# p[3] = 90
# p[4] = -12
# p[5] = -40
# p[6] = 0
# p[7] = 1
s(x) = s_logit(x, alpha, beta, X, xi)
# println(s_logit_jac(p, alpha, beta, X, xi))
ForwardDiff.jacobian(s, p)

# ForwardDiff.gradient(fp, p)


# fp(x) = fp_logit_2(x, alpha, beta, X, xi, Br)
# println(fp_logit_2_jac(p, alpha, beta, X, xi, Br))
# ForwardDiff.gradient(fp, p)

fp(x) = fp_logit(x, alpha, beta, X, xi, Br)
J_ex = J_fp_logit(p, alpha, beta, X, xi, Br)
println(J_ex)
# println(maximum(abs.(ForwardDiff.jacobian(fp, p) .- J_ex)))
println(ForwardDiff.jacobian(fp, p))
# println(inv(J_ex))

[0.0008727277308654773 -7.605723129685889e-6 -1.8863166935850017e-7 -1.3938099869248953e-6 -6.246622984529345e-6 -0.0015284982482105759 -0.00020685974334824474; -7.605723129685891e-6 0.006400040770708964 -1.3938099869248953e-6 -1.0298940184637844e-5 -4.6156647661556936e-5 -0.011294159303145172 -0.001528498248210576; -1.89518735976878e-7 -1.4003645719315781e-6 0.00032280922990471746 -1.0317339910446653e-6 -2.308810856478474e-6 -0.0005649473897043272 -7.645731499942015e-5; -1.4003645719315784e-6 -1.0347372380957437e-5 -1.0317339910446657e-6 0.0023786637026706006 -1.7059932940339566e-5 -0.004174427955469709 -0.0005649473897043273; 1.252234578345595e-5 9.252831548416369e-5 4.606713568973342e-6 3.403926499284905e-5 -0.02126277969578708 0.018569136311739884 0.0025130593222085864; 0.0030641169862345484 0.022640932304973393 0.0011272256445798898 0.00832913352395407 0.018569136311739877 -0.6776819407849809 0.6149253414458312; 0.00041468314020216867 0.0030641169862345484 0.00015255340188079274 0

In [64]:
nI = 20
nJ = 3
nN = 5

delta_1 = zeros(3)
X_1 = zeros(3,nN)
sigma_1 = 0.0 .* I(size(X,2)).* 0.1
zeta_1 = zeros(nN, nI)

delta_2 = zeros(3)
delta_2[1] = 40
delta_2[2] = 20
delta_2[3] = 20
X_2 = zeros(nJ,nN)
sigma_2 = 0.0 .* I(size(X,2)).* 0.1
zeta_2 = zeros(nN, nI)

sigma_3 = 0.1 .* I(nN)
delta_3 = zeros(nJ)
X_3a = zeros(nJ,nN)

rng = MersenneTwister(1234)
zeta_3 = randn(rng, nN, nI)

rng = MersenneTwister(1234)
X_3b = randn(rng, size(X_3a))
X_3c = X_3b .* 10
X_3d = abs.(X_3c)
    

# function u_bar(delta, X, sigma, zeta)
    
#     u_bar = zeros(size(zeta,2), size(delta,1))
    
#     for i in 1:size(u_bar, 1)
#         for j in 1:size(u_bar, 2)
#             u_bar[i,j] = delta[j] + X[j,:]' * sigma * zeta[:,i]
#         end
#     end
    
#     return u_bar
# end

# function pr(u_bar)
    
#     p = zeros(size(u_bar,1), size(u_bar,2))
    
#     for i in 1:size(p,1)
#         for j in 1:size(p,2)
#             p[i,j] = exp(u_bar[i,j])/(1 + sum(exp.(u_bar[i,:])))
#         end
#     end
    
#     return p
# end

function pr(delta::AbstractVector{T}, X, sigma, zeta) where T
    
    ubar = zeros(T, size(zeta,2), size(delta,1))
    
    for i in 1:size(ubar, 1)
        for j in 1:size(ubar, 2)
            ubar[i,j] = delta[j] + X[j,:]' * sigma * zeta[:,i]
        end
    end
    
    p = zeros(T, size(ubar,1), size(ubar,2))
    
    for i in 1:size(p,1)
        for j in 1:size(p,2)
            p[i,j] = exp(ubar[i,j])/(1 + sum(exp.(ubar[i,:])))
        end
    end
    
    return p
end




function sHat(delta::AbstractVector{T}, X, sigma, zeta) where T
#     ubar = zeros(size())
#     ubar = u_bar(delta, X, sigma, zeta)
#     p = pr(ubar)
    
    p = pr(delta, X, sigma, zeta)
    
    s = zeros(T, size(delta))
    
    for j in 1:length(s)
        s[j] = 1/size(zeta,2) * sum(p[:,j])
    end
    
    return s
end

s1 = sHat(delta_1, X_1, sigma_1, zeta_1)
s2 = sHat(delta_2, X_2, sigma_2, zeta_2)
s3a = sHat(delta_3, X_3a, sigma_3, zeta_3)
s3b = sHat(delta_3, X_3b, sigma_3, zeta_3)
s3c = sHat(delta_3, X_3c, sigma_3, zeta_3)
s3d = sHat(delta_3, X_3d, sigma_3, zeta_3)

println("s1: ", s1)
println("s2: ", s2)
println("s3a: ", s3a)
println("s3b: ", s3b)
println("s3c: ", s3c)
println("s3d: ", s3d)

s1: [0.25, 0.25, 0.25]
s2: [0.999999995877693, 2.0611536139418496e-9, 2.0611536139418496e-9]
s3a: [0.25, 0.25, 0.25]
s3b: [0.25396627272710803, 0.23546917816783988, 0.2598787383914339]
s3c: [0.302761733713778, 0.18340110893056388, 0.3631032237834327]
s3d: [0.20718104155891687, 0.1964933641693537, 0.23876547586390232]


In [66]:
function sHat_jac(delta, X, sigma, zeta)
    p = pr(delta, X, sigma, zeta)
    
    s = sHat(delta, X, sigma, zeta)
    
    sj = zeros(size(delta,1), size(delta,1))
    
    for j in 1:size(sj,1)
        for k in 1:size(sj,2)
            if j == k
                sj[j,k] = 1/size(zeta,2) * sum(p[:,j] .* (1 .- p[:,j]))
            else
                sj[j,k] = 1/size(zeta,2) * sum(-p[:, j] .* p[:, k])
            end
        end
    end
    
    return sj
end

function sHat_log_jac(delta, X, sigma, zeta)
    s = sHat(delta, X, sigma, zeta)
    sj = sHat_jac(delta, X, sigma, zeta)
    
    slj = zeros(size(delta,1), size(delta,1))
    
    for j in 1:size(slj, 1)
        for k in 1:size(slj,2)
            slj[j,k] = sj[j,k]/s[k]
        end
    end
    
    return slj
end

s1_jac = sHat_jac(delta_1, X_1, sigma_1, zeta_1)
println("s1_jac: ", s1_jac)
s1_log_jac = sHat_log_jac(delta_1, X_1, sigma_1, zeta_1)
println("s1_log_jac: ", s1_log_jac)

s1_jac: [0.1875 -0.0625 -0.0625; -0.0625 0.1875 -0.0625; -0.0625 -0.0625 0.1875]
s1_log_jac: [0.75 -0.25 -0.25; -0.25 0.75 -0.25; -0.25 -0.25 0.75]


In [69]:
function sHat_inv(s, X, sigma, zeta, tol = 1e-14, maxiter = 1e4)
    delta = zeros(size(X,1))
    diff = maximum(log.(s) - log.(sHat(delta, X, sigma, zeta)))
    it = 0
    while diff > tol && it < maxiter
        println("it: ", it)
        
        shat = sHat(delta, X, sigma, zeta)
        println("shat: ", shat)
        
        if diff > 1e-3
#             println("Contraction")
            inc = log.(s) - log.(shat)
        else
            println("Newton")
            
            slj = sHat_log_jac(delta, X, sigma, zeta)
            println("slj: ", slj)
            inc = -slj \ log.(shat)
        end
        
        delta += inc
        
        diff = maximum(inc)
        println("diff: ", diff)
        it += 1
    end
    
    println("it: ", it)
    println("diff: ", diff)
    
    
    return delta
end

println("s1_inv: ", sHat_inv(s1, X_1, sigma_1, zeta_1))
println("s2_inv: ", sHat_inv(s2, X_2, sigma_2, zeta_2))
println("s3a_inv: ", sHat_inv(s3a, X_3a, sigma_3, zeta_3))
println("s3b_inv: ", sHat_inv(s3b, X_3b, sigma_3, zeta_3))
println("s3c_inv: ", sHat_inv(s3c, X_3c, sigma_3, zeta_3))
println("s3d_inv: ", sHat_inv(s3d, X_3d, sigma_3, zeta_3))

it: 0
shat: [0.2421977436718322, 0.234936274771476, 0.2521261601141766]
diff: 0.062146611676850094
it: 1
shat: [0.24483603326713324, 0.24297054199238338, 0.2461513678034505]
diff: 0.02852070823900088
it: 2
shat: [0.24575903579475758, 0.24554800092018272, 0.24577532306804653]
diff: 0.017968466643353276
it: 3
shat: [0.24646376869528763, 0.2466784035804072, 0.24632402426388908]
diff: 0.014813076835250794
it: 4
shat: [0.2470732394329507, 0.24735779546318343, 0.2469405203400833]
diff: 0.012313418572996948
it: 5
shat: [0.24758834046351932, 0.24785115637970667, 0.2474786964842295]
diff: 0.010136414369344537
it: 6
shat: [0.24801566698833377, 0.24823905877993685, 0.24792625103522598]
diff: 0.008329590779894813
it: 7
shat: [0.24836752362301578, 0.24855288070046097, 0.24829442443300923]
diff: 0.006845680561944567
it: 8
shat: [0.24865661189749663, 0.2488093354899338, 0.2485966895988645]
diff: 0.0056290550494195735
it: 9
shat: [0.2488940891833683, 0.2490197094856847, 0.24884488510780223]
diff: 0.00

diff: 0.00888894741724755
it: 110
shat: [0.9912280660892923, 2.0430733190827162e-9, 2.0430733190827162e-9]
diff: 0.008810629682154765
it: 111
shat: [0.9913043437396261, 2.0432305390380077e-9, 2.0432305390380077e-9]
diff: 0.008733679968756292
it: 112
shat: [0.9913793062580576, 2.0433850483044232e-9, 2.0433850483044232e-9]
diff: 0.008658062743114654
it: 113
shat: [0.9914529873659177, 2.0435369163867912e-9, 2.0435369163867912e-9]
diff: 0.008583743691392698
it: 114
shat: [0.9915254196414411, 2.0436862104338693e-9, 2.0436862104338693e-9]
diff: 0.008510689667908815
it: 115
shat: [0.9915966345677962, 2.0438329953372963e-9, 2.0438329953372963e-9]
diff: 0.008438868645864765
it: 116
shat: [0.9916666625787122, 2.0439773338256674e-9, 2.0439773338256674e-9]
diff: 0.008368249670517969
it: 117
shat: [0.9917355331018444, 2.044119286553904e-9, 2.044119286553904e-9]
diff: 0.00829880281469497
it: 118
shat: [0.9918032746000066, 2.0442589121882278e-9, 2.0442589121882278e-9]
diff: 0.0082304991365163
it: 119

shat: [0.9962121171054289, 2.0533462138890376e-9, 2.0533462138890376e-9]
diff: 0.0037950709685539152
it: 261
shat: [0.9962264109875886, 2.0533756757760353e-9, 2.0533756757760353e-9]
diff: 0.0037807228399060477
it: 262
shat: [0.9962405973969496, 2.05340491614508e-9, 2.05340491614508e-9]
diff: 0.0037664827954770885
it: 263
shat: [0.9962546775410721, 2.0534339374851416e-9, 2.0534339374851416e-9]
diff: 0.0037523496185514205
it: 264
shat: [0.9962686526094924, 2.0534627422480423e-9, 2.0534627422480423e-9]
diff: 0.0037383221106073736
it: 265
shat: [0.9962825237740586, 2.053491332849133e-9, 2.053491332849133e-9]
diff: 0.003724399090982259
it: 266
shat: [0.9962962921892571, 2.0535197116679914e-9, 2.0535197116679914e-9]
diff: 0.0037105793965359624
it: 267
shat: [0.9963099589925357, 2.0535478810490754e-9, 2.0535478810490754e-9]
diff: 0.003696861881326896
it: 268
shat: [0.9963235253046131, 2.053575843302359e-9, 2.053575843302359e-9]
diff: 0.003683245416297609
it: 269
shat: [0.9963369922297892, 2.0

shat: [0.9975728114216793, 2.056150813908012e-9, 2.056150813908012e-9]
diff: 0.0024301348532916067
it: 409
shat: [0.9975786883816206, 2.0561629272252848e-9, 2.0561629272252848e-9]
diff: 0.002424243611507393
it: 410
shat: [0.9975845369504521, 2.056174982024119e-9, 2.056174982024119e-9]
diff: 0.002418380864281644
it: 411
shat: [0.9975903573334093, 2.0561869787275335e-9, 2.0561869787275335e-9]
diff: 0.0024125464053845747
it: 412
shat: [0.9975961497337562, 2.056198917754493e-9, 2.056198917754493e-9]
diff: 0.002406740030565402
it: 413
shat: [0.9976019143528065, 2.0562107995199306e-9, 2.0562107995199306e-9]
diff: 0.0024009615375391036
it: 414
shat: [0.9976076513899472, 2.0562226244348174e-9, 2.0562226244348174e-9]
diff: 0.0023952107259549112
it: 415
shat: [0.9976133610426626, 2.0562343929061923e-9, 2.0562343929061923e-9]
diff: 0.002389487397381445
it: 416
shat: [0.9976190435065556, 2.056246105337228e-9, 2.056246105337228e-9]
diff: 0.002383791355276229
it: 417
shat: [0.9976246989753706, 2.056

diff: 0.001737619898541709
it: 573
shat: [0.9982668936318044, 2.0575814239696856e-9, 2.0575814239696856e-9]
diff: 0.001734605812208291
it: 574
shat: [0.9982698920785966, 2.0575876042291493e-9, 2.0575876042291493e-9]
diff: 0.0017316021642778813
it: 575
shat: [0.9982728801680596, 2.0575937631405685e-9, 2.0575937631405685e-9]
diff: 0.0017286089006178056
it: 576
shat: [0.9982758579537658, 2.057599900814362e-9, 2.057599900814362e-9]
diff: 0.001725625967470279
it: 577
shat: [0.998278825488919, 2.0576060173601924e-9, 2.0576060173601924e-9]
diff: 0.0017226533114467202
it: 578
shat: [0.9982817828263569, 2.057612112886967e-9, 2.057612112886967e-9]
diff: 0.0017196908795291677
it: 579
shat: [0.9982847300185542, 2.0576181875028466e-9, 2.0576181875028466e-9]
diff: 0.001716738619054872
it: 580
shat: [0.9982876671176282, 2.0576242413152367e-9, 2.0576242413152367e-9]
diff: 0.001713796477734911
it: 581
shat: [0.9982905941753377, 2.057630274430836e-9, 2.057630274430836e-9]
diff: 0.001710864403630552
it: 

diff: 0.0014825799602230653
it: 672
shat: [0.9985207059429625, 2.0581045701342428e-9, 2.0581045701342428e-9]
diff: 0.001480385170435028
it: 673
shat: [0.9985228910093359, 2.0581090738917155e-9, 2.0581090738917155e-9]
diff: 0.0014781968693109563
it: 674
shat: [0.9985250696300856, 2.0581135643637654e-9, 2.0581135643637654e-9]
diff: 0.0014760150281212248
it: 675
shat: [0.9985272418336906, 2.058118041609097e-9, 2.058118041609097e-9]
diff: 0.0014738396183004423
it: 676
shat: [0.9985294076484611, 2.0581225056860543e-9, 2.0581225056860543e-9]
diff: 0.0014716706114562051
it: 677
shat: [0.9985315671025422, 2.0581269566526527e-9, 2.0581269566526527e-9]
diff: 0.0014695079793618504
it: 678
shat: [0.9985337202239135, 2.058131394566571e-9, 2.058131394566571e-9]
diff: 0.0014673516939502917
it: 679
shat: [0.9985358670403905, 2.0581358194851306e-9, 2.0581358194851306e-9]
diff: 0.0014652017273277806
it: 680
shat: [0.998538007579626, 2.05814023146533e-9, 2.05814023146533e-9]
diff: 0.0014630580517605812
i

shat: [0.9987293478524354, 2.0585346131617473e-9, 2.0585346131617473e-9]
diff: 0.00127145598819669
it: 784
shat: [0.9987309603499291, 2.058537936766796e-9, 2.058537936766796e-9]
diff: 0.0012698414404788139
it: 785
shat: [0.9987325687599772, 2.058541251946998e-9, 2.058541251946998e-9]
diff: 0.0012682309879951791
it: 786
shat: [0.9987341730981009, 2.0585445587343287e-9, 2.0585445587343287e-9]
diff: 0.001266624615194445
it: 787
shat: [0.9987357733797437, 2.0585478571606377e-9, 2.0585478571606377e-9]
diff: 0.0012650223065868718
it: 788
shat: [0.9987373696202717, 2.0585511472575832e-9, 2.0585511472575832e-9]
diff: 0.0012634240467725988
it: 789
shat: [0.998738961834972, 2.058554429056683e-9, 2.058554429056683e-9]
diff: 0.0012618298204221377
it: 790
shat: [0.998740550039056, 2.0585577025892818e-9, 2.0585577025892818e-9]
diff: 0.0012602396122877526
it: 791
shat: [0.9987421342476579, 2.0585609678865766e-9, 2.0585609678865766e-9]
diff: 0.0012586534071985511
it: 792
shat: [0.9987437144758364, 2.0

it: 870
shat: [0.998855831122684, 2.058795314612397e-9, 2.058795314612397e-9]
diff: 0.0011448198159342837
it: 871
shat: [0.9988571387395471, 2.0587980098116328e-9, 2.0587980098116328e-9]
diff: 0.0011435107020789147
it: 872
shat: [0.9988584433709833, 2.058800698857443e-9, 2.058800698857443e-9]
diff: 0.0011422045787769796
it: 873
shat: [0.998859745027205, 2.0588033817708797e-9, 2.0588033817708797e-9]
diff: 0.0011409014357930403
it: 874
shat: [0.9988610437183788, 2.0588060585728993e-9, 2.0588060585728993e-9]
diff: 0.0011396012629337623
it: 875
shat: [0.998862339454624, 2.058808729284348e-9, 2.058808729284348e-9]
diff: 0.001138304050062544
it: 876
shat: [0.9988636322460137, 2.05881139392601e-9, 2.05881139392601e-9]
diff: 0.001137009787075982
it: 877
shat: [0.9988649221025764, 2.058814052518538e-9, 2.058814052518538e-9]
diff: 0.0011357184639273566
it: 878
shat: [0.9988662090342942, 2.0588167050825057e-9, 2.0588167050825057e-9]
diff: 0.0011344300706142008
it: 879
shat: [0.9988674930511044, 2

shat: [0.9989680041378812, 2.0590265204290117e-9, 2.0590265204290117e-9]
diff: 0.0010325246141890763
it: 966
shat: [0.998969068046891, 2.0590287133089217e-9, 2.0590287133089217e-9]
diff: 0.0010314596066624715
it: 967
shat: [0.9989701297645335, 2.0590309016720843e-9, 2.0590309016720843e-9]
diff: 0.0010303967938973813
it: 968
shat: [0.9989711892975717, 2.0590330855324463e-9, 2.0590330855324463e-9]
diff: 0.001029336169114714
it: 969
shat: [0.9989722466527415, 2.0590352649038844e-9, 2.0590352649038844e-9]
diff: 0.0010282777255667952
it: 970
shat: [0.998973301836751, 2.0590374398002294e-9, 2.0590374398002294e-9]
diff: 0.001027221456529759
it: 971
shat: [0.99897435485628, 2.059039610235243e-9, 2.059039610235243e-9]
diff: 0.0010261673553131345
it: 972
shat: [0.9989754057179822, 2.0590417762226487e-9, 2.0590417762226487e-9]
diff: 0.0010251154152456365
it: 973
shat: [0.9989764544284834, 2.059043937776097e-9, 2.059043937776097e-9]
diff: 0.0010240656296903175
it: 974
shat: [0.9989775009943822, 2.

In [74]:
delta_1 = ones(3)
delta_2 = 2
delta_3 = 40
X_1 = 10 .* randn(3,nN)
sigma_1 = 0.1 .* I(size(X,2))
zeta_1 = randn(nN, 10000)

shl(x) = log.(sHat(x, X_1, sigma_1, zeta_1))
println(sHat_log_jac(delta_1, X_1, sigma_1, zeta_1))
ForwardDiff.jacobian(shl, delta_1)

[0.018711586165667322 -0.014308007489740912 -0.009412975082081205; -0.007685054697336535 0.06917905445358144 -0.03352186817963458; -0.006185213021340247 -0.041009848259149044 0.05981233793995599]


3×3 Array{Float64,2}:
  0.0187116   -0.00768505  -0.00618521
 -0.014308     0.0691791   -0.0410098
 -0.00941298  -0.0335219    0.0598123

In [ ]:
X
X'